In [1]:
#Based losely on: https://medium.com/@sabber/classifying-yelp-review-comments-using-lstm-and-word-embeddings-part-1-eb2275e4066b

In [84]:
# Keras
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation, GlobalAveragePooling1D
from keras.layers.embeddings import Embedding

from keras.optimizers import SGD

from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.utils import np_utils


import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)
# Others
import nltk
import string
import numpy as np
import pandas as pd
from nltk.corpus import stopwords

from sklearn.manifold import TSNE

In [14]:
colors_df=pd.read_csv('color_data_final.csv')
colors_df = colors_df.sample(frac=1).reset_index(drop=True)#shuffle dataframe - very important to learn all classes at the same time

In [30]:
np.max([len(nltk.word_tokenize(x)) for x in colors_df.context])

27

In [31]:
print("Dimensions of data: ", colors_df.shape)
maxlen = np.max([len(nltk.word_tokenize(x)) for x in colors_df.context])
print("Maximum context dimension: ", maxlen)
colors_df.head(3)

Dimensions of data:  (4235, 7)
Maximum context dimension:  27


,Unnamed: 0,name,context,code,red,green,blue
0,467,black,like saying red ninjas blue ninjas green ninja...,"(0, 0, 0)",0,0,0
1,3035,yellow,varied gathering products busy far sunlit pano...,"(255, 255, 0)",255,255,0
2,695,blue,real difference natural supernatural flew sky ...,"(0, 0, 255)",0,0,255


In [116]:
vocabulary_size = 20000
t = Tokenizer(num_words= vocabulary_size)
t.fit_on_texts(colors_df.context)
sequences = t.texts_to_sequences(colors_df.context)
#print(sequences)
data = pad_sequences(sequences, maxlen=maxlen)

dataY = np.column_stack([colors_df.red/255, colors_df.green/255, colors_df.blue/255])

In [111]:
def create_conv_model():
    model = Sequential()
    model.add(Embedding(vocabulary_size, 100, input_length=maxlen))
    #model.add(Dropout(0.1))
    model.add(Conv1D(64, 2))# activation='relu'))
    model.add(Conv1D(64, 3))# activation='relu'))
    model.add(Conv1D(64, 2))# activation='relu'))
    #model.add(MaxPooling1D(pool_size=2)) # sliding a kernel of 4 accross the input (for 2D is (4,4)) etc.
    #model.add(Conv1D(512, 3, activation='relu'))
    #model.add(GlobalAveragePooling1D())
    #model.add(LSTM(256, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))
    #model.add(LSTM(100, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))# return_sequences=True,
    model.add(LSTM(100, dropout=0.1))#, recurrent_dropout=0.2))
    #model.add(Dense(128))#, activation='relu'))# sigmoid#relu
    model.add(Dense(3, activation='sigmoid'))#softmax
    
    adam = keras.optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
    model.compile(loss='binary_crossentropy', optimizer=adam,    metrics=['accuracy'])
    
    #from https://keras.io/callbacks/
    early_stopping = EarlyStopping(monitor='val_acc', min_delta=0, patience=20, verbose=0)#, restore_best_weights=False)# monitor='val_loss'
    checkpointer = ModelCheckpoint('model_best_conv_{val_acc:.2f}.hdf5', monitor='val_acc', verbose=1, save_best_only=True)#weights.{epoch:02d}-{val_loss:.2f}.hdf5 #monitor='val_loss'#{val_loss:.2f}
    model.summary()
    return model, early_stopping, checkpointer

In [112]:
model, early_stopping, checkpointer = create_conv_model()
#model_conv.fit(data, np.array(labels), validation_split=0.4, epochs = 3)
#model.fit(data, dataY, validation_split=0.2, epochs = 50)


history = model.fit(data, dataY,
                    epochs=120,
                    batch_size=32,
                    validation_split=0.1,
                   shuffle=True, callbacks=[early_stopping, checkpointer])
model.save('model_conv_final.hdf5')


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_47 (Embedding)     (None, 27, 100)           2000000   
_________________________________________________________________
conv1d_41 (Conv1D)           (None, 26, 64)            12864     
_________________________________________________________________
conv1d_42 (Conv1D)           (None, 24, 64)            12352     
_________________________________________________________________
conv1d_43 (Conv1D)           (None, 23, 64)            8256      
_________________________________________________________________
lstm_83 (LSTM)               (None, 100)               66000     
_________________________________________________________________
dense_69 (Dense)             (None, 3)                 303       
Total params: 2,099,775
Trainable params: 2,099,775
Non-trainable params: 0
_________________________________________________________________



Epoch 00035: val_acc did not improve
Epoch 36/120
3811/3811 [==============================] - 13s 3ms/step - loss: 0.1249 - acc: 0.8658 - val_loss: 1.0777 - val_acc: 0.6305

Epoch 00036: val_acc did not improve
Epoch 37/120
3811/3811 [==============================] - 13s 3ms/step - loss: 0.1235 - acc: 0.8674 - val_loss: 1.0809 - val_acc: 0.6344

Epoch 00037: val_acc did not improve
Epoch 38/120
3811/3811 [==============================] - 13s 3ms/step - loss: 0.1221 - acc: 0.8659 - val_loss: 1.0885 - val_acc: 0.6321

Epoch 00038: val_acc did not improve
Epoch 39/120
3811/3811 [==============================] - 13s 3ms/step - loss: 0.1204 - acc: 0.8673 - val_loss: 1.0839 - val_acc: 0.6329

Epoch 00039: val_acc did not improve
Epoch 40/120
3811/3811 [==============================] - 13s 3ms/step - loss: 0.1194 - acc: 0.8683 - val_loss: 1.0983 - val_acc: 0.6281

Epoch 00040: val_acc did not improve
Epoch 41/120
3811/3811 [==============================] - 13s 3ms/step - loss: 0.1181 -

In [44]:
def get_model1():
    model = Sequential()
    model.add(Embedding(20000, 100, input_length=maxlen))
    model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(3, activation='sigmoid'))
    
    adam = keras.optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
    model.compile(loss='binary_crossentropy', optimizer=adam,    metrics=['accuracy'])
    
    #from https://keras.io/callbacks/
    early_stopping = EarlyStopping(monitor='val_acc', min_delta=0, patience=7, verbose=0)#, restore_best_weights=False)# monitor='val_loss'
    checkpointer = ModelCheckpoint('model_best_word_{val_acc:.2f}.hdf5', monitor='val_acc', verbose=1, save_best_only=True)#weights.{epoch:02d}-{val_loss:.2f}.hdf5 #monitor='val_loss'#{val_loss:.2f}
    model.summary()
    return model, early_stopping, checkpointer
    #model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [45]:
model, early_stopping, checkpointer = get_model1()
#model_conv.fit(data, np.array(labels), validation_split=0.4, epochs = 3)
#model.fit(data, dataY, validation_split=0.2, epochs = 50)


history = model.fit(data, dataY,
                    epochs=120,
                    batch_size=32,
                    validation_split=0.1,
                   shuffle=True, callbacks=[early_stopping, checkpointer])
model.save('model_word_final.hdf5')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_14 (Embedding)     (None, 27, 100)           2000000   
_________________________________________________________________
lstm_24 (LSTM)               (None, 100)               80400     
_________________________________________________________________
dense_22 (Dense)             (None, 3)                 303       
Total params: 2,080,703
Trainable params: 2,080,703
Non-trainable params: 0
_________________________________________________________________
Train on 3811 samples, validate on 424 samples
Epoch 1/120
3811/3811 [==============================] - 16s 4ms/step - loss: 0.6886 - acc: 0.4930 - val_loss: 0.6869 - val_acc: 0.4733

Epoch 00001: val_acc improved from -inf to 0.47327, saving model to model_best_word_0.47.hdf5
Epoch 2/120
3811/3811 [==============================] - 14s 4ms/step - loss: 0.6827 - acc: 0.4932 - val_loss: 0.6853 

Epoch 35/120
3811/3811 [==============================] - 14s 4ms/step - loss: 0.2055 - acc: 0.8484 - val_loss: 0.8900 - val_acc: 0.6399

Epoch 00035: val_acc improved from 0.63758 to 0.63994, saving model to model_best_word_0.64.hdf5
Epoch 36/120
3811/3811 [==============================] - 14s 4ms/step - loss: 0.1984 - acc: 0.8515 - val_loss: 0.9364 - val_acc: 0.6336

Epoch 00036: val_acc did not improve
Epoch 37/120
3811/3811 [==============================] - 14s 4ms/step - loss: 0.1966 - acc: 0.8527 - val_loss: 0.8866 - val_acc: 0.6392

Epoch 00037: val_acc did not improve
Epoch 38/120
3811/3811 [==============================] - 14s 4ms/step - loss: 0.1931 - acc: 0.8527 - val_loss: 0.9499 - val_acc: 0.6329

Epoch 00038: val_acc did not improve
Epoch 39/120
3811/3811 [==============================] - 14s 4ms/step - loss: 0.1877 - acc: 0.8555 - val_loss: 0.9395 - val_acc: 0.6384

Epoch 00039: val_acc did not improve
Epoch 40/120
3811/3811 [==============================] - 14s 4ms

In [117]:
word_list = []
for word, i in t.word_index.items():
    word_list.append(word)

In [118]:
## Get weights
conv_embds = model.layers[0].get_weights()[0]
## Plotting function
def plot_words(data, start, stop, step):
    trace = go.Scatter(
        x = data[start:stop:step,0], 
        y = data[start:stop:step, 1],
        mode = 'markers',
        text= word_list[start:stop:step]
    )
    layout = dict(title= 't-SNE 1 vs t-SNE 2',
                  yaxis = dict(title='t-SNE 2'),
                  xaxis = dict(title='t-SNE 1'),
                  hovermode= 'closest')
    fig = dict(data = [trace], layout= layout)
    py.iplot(fig)
## Visualize words in two dimensions 
conv_tsne_embds = TSNE(n_components=2).fit_transform(conv_embds)
plot_words(conv_tsne_embds, 0, 2000, 1)